# 1. Carregamento e preparação dos dados:

Nesta etapa, carregamos o conjunto de dados utilizando a biblioteca Pandas, verificando as colunas e informações relevantes. Criamos identificadores únicos para cada transação e convertemos esses IDs para valores numéricos usando LabelEncoder. Em seguida, classificamos os preços dos produtos em três categorias: "alto", "baixo" e "ideal", com base na comparação entre o preço atual e o preço histórico.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Carregar o dataset
df = pd.read_csv('retail_price.csv')

# Criar um identificador único combinando várias colunas
df['unique_id'] = df['product_id'] + '_' + df['month_year'].astype(str) + '_' + df['qty'].astype(str)

# Transformar o 'unique_id' em IDs numéricos únicos
label_encoder = LabelEncoder()
df['unique_id_numeric'] = label_encoder.fit_transform(df['unique_id'])

# Criar uma coluna alvo baseada em 'unit_price' e 'lag_price'
df['price_category'] = df.apply(
    lambda row: 'alto' if row['unit_price'] > row['lag_price'] else ('baixo' if row['unit_price'] < row['lag_price'] else 'ideal'),
    axis=1
)

# Selecionar as colunas relevantes para o modelo
features = ['qty', 'total_price', 'freight_price', 'product_photos_qty', 'ps1', 'fp1', 'comp_1']
target = 'price_category'

# Preparar os dados
X = df[features]
y = df[target]

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# 2. Treinamento do modelo:

Selecionamos as características mais relevantes (como quantidade vendida, preço total, preço do frete, etc.) para treinar o modelo de machine learning. Dividimos os dados em conjuntos de treino e teste, normalizamos os dados para garantir escalabilidade, e utilizamos o algoritmo Random Forest para treinar o modelo de classificação, adequando-o para a tarefa de análise de preços.

In [3]:
# Instanciar o modelo de Random Forest Classifier
model = RandomForestClassifier()

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

import pickle
filename = 'modelo.pk1'
pickle.dump(model, open(filename, 'wb'))

# Avaliar o modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[15  4 23]
 [ 1 15 17]
 [ 8  7 46]]
              precision    recall  f1-score   support

        alto       0.62      0.36      0.45        42
       baixo       0.58      0.45      0.51        33
       ideal       0.53      0.75      0.63        61

    accuracy                           0.56       136
   macro avg       0.58      0.52      0.53       136
weighted avg       0.57      0.56      0.54       136



# 3. Análise de Produto Específico:

Por último, selecionamos aleatoriamente um produto do conjunto de teste e realizamos a previsão do seu preço utilizando o modelo treinado. Analisamos se o preço está alto, baixo ou ideal e calculamos o preço ideal com base na média dos preços históricos do produto. Exibimos os resultados, incluindo uma recomendação de ajuste de preço caso necessário, fornecendo insights precisos para a otimização de preços no setor varejista.

In [4]:
# Escolher um produto aleatório do conjunto de teste para análise
random_index = np.random.choice(X_test.shape[0])
produto_especifico = X_test[random_index].reshape(1, -1)

# Fazer a previsão
predicao = model.predict(produto_especifico)

# Obter detalhes do produto analisado
produto_analise = df.iloc[random_index]
produto_nome = produto_analise['product_id']
produto_valor = produto_analise['unit_price']

# Calcular o preço ideal baseado na média dos preços anteriores do produto
historico_precos = df[df['product_id'] == produto_nome]['unit_price']
preco_ideal = historico_precos.mean()

# Verificar a previsão e fornecer recomendação de preço
if predicao[0] == 'alto':
    recomendacao = f"O preço ideal seria ao redor de ${preco_ideal:.2f}"
elif predicao[0] == 'baixo':
    recomendacao = f"O preço ideal seria ao redor de ${preco_ideal:.2f}"
else:
    recomendacao = ""

# Exibir o resultado com o nome do produto, o valor analisado e a recomendação de preço
print(f'O preço do produto {produto_nome} no valor de ${produto_valor:.2f} está {predicao[0]}. {recomendacao}')


O preço do produto consoles1 no valor de $36.20 está ideal. 
